In [1]:
from context import *

`circuit.py` contains **three classes**: `CliffordGate`, `CliffordLayer` and `CliffordCircuit`

# CliffordGate

## Basic Usage

`CliffordGate` class represents the Clifford gates. 
- **Attribute**:
    - qubits: a tuple containing the number of qubits that it acts on 
    - n: number of qubits it acts on
    - generator(Pauli): the generator for $C_{\pi/4}$ rotation. `Default: None`
    - forward_map(CliffordMap): forward Clifford Map. `Default: None`
    - backward_map(CliffordMap): backward Clifford Map. `Default: None`
- **Methods**:
    - `CliffordGate.set_generator(Pauli)`
    - `CliffordGate.set_forward_map(CliffordMap)`
    - `CliffordGate.set_backward_map(CliffordMap)` **TODO: we need to add a handle to prevent user set conflict Clifford maps**
    - `CliffordGate.copy()`: return a copy of the Clifford gate
    - `CliffordGate.independent_from(Other_gate)`: if there is no overlap between acting qubits, then they are independent.
    - `CliffordGate.compile()`: build (forward&backward) Clifford map representation. If generator is given, then it will be converted to forward/backward map; if forward map is given, the backward map will be calculated; if backward map is given, the forward map will be calculated
    - `CliffordGate.forward(obj)`: forward transformation of the object. `obj` can be Pauli, PauliList, PauliPolynomial, StabilizerState.

CliffordGate is the low-level API(class) for Clifford gates. One can create a **null** Clifford gate by sepecifying what are the qubits this gate will act on:

Example: `circuit.CliffordGate(1,2)` will create a null gate acting on qubit 1, and 2. Null gate does not contain any `forward_map` or `backward_map`

In [2]:
gate = circuit.CliffordGate(1,2)

In [3]:
print("gate acts on: {}, and there are {} qubits the gates will act on.".format(
gate.qubits, gate.n
))

gate acts on: (1, 2), and there are 2 qubits the gates will act on.


There are three ways to equip the null gate with Clifford map:
1. One can use `gate.set_generator(Pauli)`. This will create a $\pi/2$ rotation generated by the pauli string.
2. One can use `gate.set_forward_map(CliffordMap)`
3. One can use `gate.set_backward_map(CliffordMap)`

In [4]:
gate.set_generator(paulialg.pauli("XX"))

In [5]:
gate.set_forward_map(stabilizer.random_clifford_map(2))

In [6]:
gate.set_backward_map(stabilizer.random_clifford_map(2))

`gate.compile()` will construct both `forward_clifford_map` and `backward_clifford_map`. Compilation is not necessary!

`gate.copy()` will return a copy of the `CliffordGate`

One can apply Clifford Gate $U$ or $U^{\dagger}$ to: Pauli strings, Stabilizer States by `gate.forward(obj)` and `gate.backward(obj)`

In [7]:
psi = stabilizer.random_clifford_state(5)

In [8]:
psi

StabilizerState(
   +XZYYI
   +YZXYZ
   +IIXZX
   +IZIII
   +ZIXYY)

In [9]:
gate = circuit.CliffordGate(1,2)
gate.set_generator(paulialg.pauli("XX"))

In [10]:
gate.forward(psi)

StabilizerState(
   +XZYYI
   -YYIYZ
   +IIXZX
   -IYXII
   +ZIXYY)

In [11]:
psi

StabilizerState(
   +XZYYI
   -YYIYZ
   +IIXZX
   -IYXII
   +ZIXYY)

We see the wavefunction $|\psi\rangle$ has been changed **in-place**. And the gate only changes qubit 1&2.

In [12]:
gate.backward(psi)

StabilizerState(
   +XZYYI
   +YZXYZ
   +IIXZX
   +IZIII
   +ZIXYY)

If two Clifford gate acting on different qubits, then they are independent. We can check dependency by `gate.independent_from(other_gate)`

In [13]:
gate1 = circuit.CliffordGate(0,1)
gate2 = circuit.CliffordGate(2,3)
print(gate1.independent_from(gate2))

True


In [14]:
gate1 = circuit.CliffordGate(0,1)
gate2 = circuit.CliffordGate(1,2)
print(gate1.independent_from(gate2))

False


# CliffordLayer

`CliffordLayer` has the following attributes:
- gates(list): contains a list of `CliffordGate`
- prev_layer: Default `None`. It will be automatically setup in `CliffordCircuit`
- next_layer: Default `None`. It will be automatically setup in `CliffordCircuit`
- forward_map
- backward_map

`CliffordLayer` can take `CliffordGates` by `layer.take(gate)`:

In [15]:
circlayer = circuit.CliffordLayer()

In [16]:
circlayer.take(circuit.CliffordGate(0,1))

In [17]:
circlayer.take(circuit.CliffordGate(1,3))

In [18]:
circlayer

|[0,1][1,3]|

In [19]:
circlayer.gates

[[0,1], [1,3]]

In [20]:
psi=stabilizer.random_clifford_state(4)
print(psi)

StabilizerState(
   +YZZI
   +ZXZI
   +ZZXZ
   +IIIZ)


In [21]:
circlayer.forward(psi)

StabilizerState(
   +YIZZ
   +XXZY
   -YYXX
   -IZIZ)

<div class="alert alert-block alert-success">
If the forward map and backward map for a gate is Null, then it will be assigned a <b> different </b>random clifford map each time when use calls forward() or backward()
</div>

# CliffordCircuit

`CliffordCircuit` is the high level API to assemble gates and layers, and it **automatically** calculate number of qubits in the system
It has attributes:
- first_layer
- last_layer
- forward_map
- backward_map

When `CliffordCircuit` is initialized, a null `CliffordLayer()` will be initiated.
`CliffordCircuit` can take `CliffordGate` by `circ.take(gate)`.

If the gate is independent from the current layer, it will be added. Otherwise, the circuit will create a new layer and add the gate.

In [22]:
circ = circuit.CliffordCircuit()

In [23]:
circ.take(circuit.CliffordGate(0,1))

CliffordCircuit(
  |[0,1]|)

In [24]:
circ.take(circuit.CliffordGate(9,10))

CliffordCircuit(
  |[0,1][9,10]|)

In [25]:
circ.N

11

In [26]:
psi = stabilizer.ghz_state(11)

In [27]:
circ.forward(psi)

StabilizerState(
   -IZIIIIIIIII
   +XIZIIIIIIII
   +IIZZIIIIIII
   +IIIZZIIIIII
   +IIIIZZIIIII
   +IIIIIZZIIII
   +IIIIIIZZIII
   +IIIIIIIZZII
   -IIIIIIIIZZZ
   +IIIIIIIIIXX
   +ZZXXXXXXXXI)

In [28]:
psi.entropy([1,2,3,4,5])

1

In [29]:
circuit.clifford_rotation_gate([0,2,3,0,7]).generator

-iYZ

In [30]:
circ = circuit.identity_circuit(5).take(circuit.clifford_rotation_gate([0,2,3,0,5])).take(circuit.clifford_rotation_gate([0,0,0,1,3,4]))

In [31]:
circ.compile(5)

CliffordCircuit(
  |[1,2][3,4]|)

In [32]:
circ.forward_map

CliffordMap(
  X0-> +XIIII
  Z0-> +ZIIII
  X1-> +IZZII
  Z1-> -IXZII
  X2-> -IYYII
  Z2-> +IIZII
  X3-> +IIIXI
  Z3-> -IIIYZ
  X4-> +IIIXY
  Z4-> +IIIIZ)

# $Z_2$ rank

In [33]:
def testz2rank(mat):
    '''
    Parameters:
    mat: int matrix - input binary matrix.
        caller must ensure that mat contains only 0 and 1.
        mat is destroyed upon output! 
    Returns:
    r: int - rank of the matrix under Z2 algebra.'''
    nr, nc = mat.shape # get num of rows and cols
    r = 0 # current row index
    for i in range(nc): # run through cols
        if r == nr: # row exhausted first
            return r # row rank is full, early return
        if mat[r, i] == 0: # need to find pivot
            found = False # set a flag
            for k in range(r + 1, nr):
                if mat[k, i]: # mat[k, i] nonzero
                    found = True # pivot found in k
                    break
            if found: # if pivot found in k
                # swap rows r, k
                for j in range(i, nc):
                    tmp = mat[k,j]
                    mat[k,j] = mat[r, j]
                    mat[r,j] = tmp
            else: # if pivot not found
                continue # done with this col
        # pivot has moved to mat[r, i], perform GE
        for j in range(r + 1, nr):
            if mat[j, i]: # mat[j, i] nonzero
                mat[j, i:] = (mat[j, i:] + mat[r, i:])%2
        r = r + 1 # rank inc
    # col exhausted, last nonvanishing row indexed by r
    return r

In [34]:
binary_mat = np.array([[0,1,0,0,1],[1,0,1,1,0],[1,1,1,1,1],[0,0,0,0,1]])

In [35]:
testz2rank(binary_mat)

3

In [36]:
binary_mat

array([[1, 0, 1, 1, 0],
       [0, 1, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0]])